In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from fastapi import FastAPI
from pydantic import BaseModel
from datetime import datetime
import uvicorn

In [7]:
class PredictionInput(BaseModel):
    date: str
    product_category: str
    region: str
    agent_id: str
    marketing_spend: float
    lead_count: int

In [8]:
# Analyzing the data
import pandas as pd

def check_data_issues(df):
    """Checks dataset for issues like missing values, invalid dates, datatypes, and duplicates."""
    print("Data Issues Report:")
    print(f"Total Rows: {len(df)}")
    
    # 1. Missing Values
    missing = df.isna().sum()
    print("\nMissing Values (isna()):")
    for col, count in missing.items():
        if count > 0:
            print(f"- {col}: {count} missing values")
    
    # 2. Invalid Categorical Values ('Null', 'N/A', empty)
    categorical_cols = ['product_category', 'region', 'agent_id']
    print("\nInvalid Categorical Values ('Null', 'N/A', empty):")
    for col in categorical_cols:
        if col in df.columns:
            invalid = df[col].isin(['Null', 'N/A', '']).sum()
            if invalid > 0:
                print(f"- {col}: {invalid} invalid values")
        else:
            print(f"- {col}: Column not found")
    
    # 3. Invalid Dates
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')
        invalid_dates = df['date'].isna().sum() + (df['date'].dt.year != 2025).sum()
        print(f"\nInvalid Dates (unparseable or not in 2025): {invalid_dates}")
    else:
        print("\nInvalid Dates: 'date' column not found")
    
    # 4. Non-Numeric Values in Numeric Columns
    numeric_cols = ['marketing_spend', 'lead_count', 'revenue']
    print("\nNon-Numeric Values in Numeric Columns:")
    for col in numeric_cols:
        if col in df.columns:
            non_numeric = pd.to_numeric(df[col], errors='coerce').isna().sum()
            if non_numeric > 0:
                print(f"- {col}: {non_numeric} non-numeric values")
            negative = (pd.to_numeric(df[col], errors='coerce') < 0).sum()
            if negative > 0:
                print(f"- {col}: {negative} negative values")
        else:
            print(f"- {col}: Column not found")
    
    # 5. Duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate Rows (duplicated()): {duplicates}")

if __name__ == "__main__":
    # Load CSV file directly
    df = pd.read_csv("/kaggle/input/forecasting-dataset/dummy_revenue_forecasting_data.csv")
    check_data_issues(df)

Data Issues Report:
Total Rows: 610998

Missing Values (isna()):
- agent_id: 22242 missing values
- marketing_spend: 1080 missing values
- lead_count: 1080 missing values
- revenue: 1080 missing values

Invalid Categorical Values ('Null', 'N/A', empty):
- region: 6602 invalid values
- agent_id: 2 invalid values

Invalid Dates (unparseable or not in 2025): 156890

Non-Numeric Values in Numeric Columns:
- marketing_spend: 12810 non-numeric values


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


- lead_count: 1080 non-numeric values
- revenue: 6562 non-numeric values

Duplicate Rows (duplicated()): 620


In [5]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df['revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
df.head(15)

,date,product_category,region,agent_id,marketing_spend,lead_count,revenue,week_start
0,2025-03-29,Cosmetic,North,A4,21000.0,170.0,75000.0,2025-03-24
1,2025-03-29,Logistic,South,A5,14000.0,60.0,43000.0,2025-03-24
2,2025-03-29,Fintech,West,A5,31000.0,70.0,89000.0,2025-03-24
3,2025-03-30,Cosmetic,North,A3,47000.0,70.0,86000.0,2025-03-24
4,2025-03-30,FMCG,West,A9,19000.0,80.0,74000.0,2025-03-24
5,2025-03-30,FMCG,East,A8,5000.0,120.0,86000.0,2025-03-24
6,2025-03-31,FMCG,East,A3,5000.0,170.0,70000.0,2025-03-31
7,2025-03-31,Insurance,South,A3,20000.0,140.0,72000.0,2025-03-31
8,2025-03-31,Fintech,South,A9,34000.0,190.0,51000.0,2025-03-31
9,2025-04-01,Insurance,East,A5,22000.0,180.0,71000.0,2025-03-31


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from lightgbm import LGBMRegressor
import joblib

# Load and clean data
df = pd.read_csv("/kaggle/input/forecast-dataset/dummy_revenue_forecasting_data.csv")  # <-- change path if needed
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['marketing_spend'] = pd.to_numeric(df['marketing_spend'], errors='coerce')
df.dropna(subset=['date', 'revenue', 'marketing_spend', 'lead_count'], inplace=True)

# Weekly aggregation
df['week_start'] = df['date'] - pd.to_timedelta(df['date'].dt.dayofweek, unit='d')
weekly = df.groupby(['week_start', 'product_category', 'region', 'agent_id']).agg({
    'revenue': 'sum',
    'marketing_spend': 'sum',
    'lead_count': 'sum'
}).reset_index()

# Lag and rolling features
weekly.sort_values(by=['agent_id', 'week_start'], inplace=True)
for lag in [1, 2]:
    weekly[f'revenue_lag_{lag}'] = weekly.groupby('agent_id')['revenue'].shift(lag)
    weekly[f'marketing_lag_{lag}'] = weekly.groupby('agent_id')['marketing_spend'].shift(lag)
    weekly[f'leads_lag_{lag}'] = weekly.groupby('agent_id')['lead_count'].shift(lag)

weekly['revenue_roll_mean_2'] = weekly.groupby('agent_id')['revenue'].shift(1).rolling(2).mean()
weekly['revenue_roll_mean_3'] = weekly.groupby('agent_id')['revenue'].shift(1).rolling(3).mean()
weekly['marketing_per_lead'] = weekly['marketing_spend'] / (weekly['lead_count'] + 1e-5)

# Date encodings
weekly['week_num'] = weekly['week_start'].dt.isocalendar().week
weekly['month'] = weekly['week_start'].dt.month

# Frequency encodings
weekly['region_freq'] = weekly['region'].map(weekly['region'].value_counts(normalize=True))
weekly['product_cat_freq'] = weekly['product_category'].map(weekly['product_category'].value_counts(normalize=True))

# Final cleanup
weekly.dropna(inplace=True)

# Feature set
features = [
    'marketing_spend', 'lead_count', 'marketing_per_lead',
    'revenue_lag_1', 'revenue_lag_2', 'revenue_roll_mean_2', 'revenue_roll_mean_3',
    'marketing_lag_1', 'marketing_lag_2', 'leads_lag_1', 'leads_lag_2',
    'week_num', 'month', 'region_freq', 'product_cat_freq'
]
target = 'revenue'

X = weekly[features]
y = np.log1p(weekly[target])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Model
model = LGBMRegressor(n_estimators=1000, learning_rate=0.03, num_leaves=50, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

print(f"✅ MAE: {mean_absolute_error(y_true, y_pred):,.2f}")
print(f"✅ R²: {r2_score(y_true, y_pred):.4f}")
print(f"✅ MAPE: {mean_absolute_percentage_error(y_true, y_pred):.4f}")

# Save artifacts
joblib.dump(model, "final_weekly_model.pkl")
joblib.dump(features, "final_weekly_features.pkl")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000281 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2833
[LightGBM] [Info] Number of data points in the train set: 2768, number of used features: 15
[LightGBM] [Info] Start training from score 15.630068
✅ MAE: 288,528.80
✅ R²: 0.9950
✅ MAPE: 0.0584


['final_weekly_features.pkl']

In [2]:
import joblib
import pandas as pd
import numpy as np

# Load model and features
model = joblib.load("final_model2.pkl")
features = joblib.load("final_features2.pkl")

# Construct correct input
row = {
    "marketing_spend": 21000.0,
    "lead_count": 170.0,
    "revenue_lag_1": 74000.0,
    "revenue_lag_2": 72000.0,
    "revenue_lag_3": 70000.0,
    "revenue_roll_mean_3": 72000.0,
    "marketing_lag_1": 19500.0,
    "marketing_lag_2": 18500.0,
    "marketing_lag_3": 18000.0,
    "leads_lag_1": 160.0,
    "leads_lag_2": 150.0,
    "leads_lag_3": 140.0,
    "weekofyear": 13,
    "month": 3,
    "product_category_freq": 0.15,
    "region_freq": 0.30,
    "agent_id_freq": 0.008,
    "product_category_mean_rev": 72000.0,
    "region_mean_rev": 68000.0,
    "agent_id_mean_rev": 71000.0
}

X_new = pd.DataFrame([row])[features]
log_pred = model.predict(X_new)[0]
forecasted_revenue = np.expm1(log_pred)

print(f"✅ Forecasted Weekly Revenue: ₹{forecasted_revenue:,.2f}")


✅ Forecasted Weekly Revenue: ₹3,298,651.15


In [3]:
df.head()

,date,product_category,region,agent_id,marketing_spend,lead_count,revenue,week
0,2025-03-29,Cosmetic,North,A4,21000.0,170.0,75000.0,2025-03-24
1,2025-03-29,Logistic,South,A5,14000.0,60.0,43000.0,2025-03-24
2,2025-03-29,Fintech,West,A5,31000.0,70.0,89000.0,2025-03-24
3,2025-03-30,Cosmetic,North,A3,47000.0,70.0,86000.0,2025-03-24
4,2025-03-30,FMCG,West,A9,19000.0,80.0,74000.0,2025-03-24


In [4]:
# Show all agents with >= 3 weeks of history
counts = weekly_df.groupby('agent_id')['week'].nunique()
valid_agents = counts[counts >= 3].index.tolist()
print("✅ Agents with at least 3 weeks of data:", valid_agents)


✅ Agents with at least 3 weeks of data: ['A1', 'A10', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']


In [15]:
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/input/forecast-dataset/dummy_revenue_forecasting_data.csv")

# Step 1: Ensure numeric revenue
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

# Step 2: Drop NaNs
df = df[df['revenue'].notna()]

# Step 3: IQR filtering
Q1 = df['revenue'].quantile(0.25)
Q3 = df['revenue'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df[(df['revenue'] >= lower_bound) & (df['revenue'] <= upper_bound)]

# Step 4: Compare counts
print("Original rows:", len(df))
print("After removing outliers:", len(df_filtered))





Original rows: 604436
After removing outliers: 604436


In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['revenue_scaled'] = scaler.fit_transform(df[['revenue']])


In [20]:
df['log_revenue'] = np.log1p(df['revenue'])


In [22]:
print("Revenue stats:")
print(weekly_df['revenue'].describe())


Revenue stats:
count    2.770000e+03
mean     1.246611e+07
std      2.872793e+06
min      2.717000e+06
25%      1.240900e+07
50%      1.328500e+07
75%      1.408075e+07
max      1.727100e+07
Name: revenue, dtype: float64


In [12]:
df.head(20)

,date,product_category,region,agent_id,marketing_spend,lead_count,revenue,week
0,2025-03-29,Cosmetic,North,A4,21000.0,170.0,75000.0,2025-03-24
1,2025-03-29,Logistic,South,A5,14000.0,60.0,43000.0,2025-03-24
2,2025-03-29,Fintech,West,A5,31000.0,70.0,89000.0,2025-03-24
3,2025-03-30,Cosmetic,North,A3,47000.0,70.0,86000.0,2025-03-24
4,2025-03-30,FMCG,West,A9,19000.0,80.0,74000.0,2025-03-24
5,2025-03-30,FMCG,East,A8,5000.0,120.0,86000.0,2025-03-24
6,2025-03-31,FMCG,East,A3,5000.0,170.0,70000.0,2025-03-31
7,2025-03-31,Insurance,South,A3,20000.0,140.0,72000.0,2025-03-31
8,2025-03-31,Fintech,South,A9,34000.0,190.0,51000.0,2025-03-31
9,2025-04-01,Insurance,East,A5,22000.0,180.0,71000.0,2025-03-31
